<a href="https://colab.research.google.com/github/robmaz22/Kaggle-competitions/blob/main/Tabular_Playground_Series_Nov_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle competition [link](https://www.kaggle.com/c/tabular-playground-series-nov-2021/data)

#1. Download dataset

In [ ]:
!pip install -q kaggle
from google.colab import files

files.upload()

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c tabular-playground-series-nov-2021

Saving kaggle.json to kaggle.json
 97% 197M/202M [00:02<00:00, 119MB/s]
100% 202M/202M [00:02<00:00, 75.0MB/s]
  0% 0.00/1.16M [00:00<?, ?B/s]
100% 1.16M/1.16M [00:00<00:00, 116MB/s]
 98% 220M/225M [00:02<00:00, 123MB/s]
100% 225M/225M [00:02<00:00, 106MB/s]


# 2. Unzip and read train dataset

In [ ]:
!unzip train.csv.zip
!unzip test.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                


In [ ]:
import pandas as pd

In [ ]:
raw_data = pd.read_csv('train.csv')
raw_data.head()

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,...,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,target
0,0,0.106643,3.59437,132.8040,3.18428,0.081971,1.18859,3.73238,2.266270,2.09959,0.012330,1.607190,-0.318058,0.560137,2.806880,1.35114,2.535930,0.197527,0.676494,1.98979,-3.842450,0.037380,0.230322,3.33055,0.009397,0.144738,3.05131,1.30362,0.033225,-0.018284,2.748210,-0.009294,-0.036271,-0.049871,0.019484,3.898460,11.2863,1.138020,3.366880,4.94446,...,-0.027551,0.019483,-0.048826,0.050748,3.729300,5.017440,4.186880,0.063342,0.121043,1.37175,4.017450,0.167613,0.039754,2.042360,-0.016614,0.107679,3.507250,0.013660,-0.097023,5.396070,0.244457,3.49184,0.113090,-0.015472,4.208790,4.106560,0.037227,-0.118814,0.067086,0.010739,1.09862,0.013331,-0.011715,0.052759,0.065400,4.211250,1.97877,0.085974,0.240496,0
1,1,0.125021,1.67336,76.5336,3.37825,0.099400,5.09366,1.27562,-0.471318,4.54594,0.037706,0.331749,0.325091,0.062040,2.262150,4.33943,-0.224999,0.233586,3.381280,1.90299,0.067874,-0.051268,0.006135,2.60444,0.103441,0.067638,4.75362,1.85552,-0.181834,0.008359,3.166340,0.011850,0.022292,0.069320,0.117109,0.315276,24.4807,1.672270,-0.409067,4.95475,...,-0.010841,0.064584,0.102548,0.093611,0.964089,0.630422,4.307340,0.091289,-0.036360,3.61767,3.103240,0.000657,0.051302,1.924620,0.123294,-0.022671,1.548120,-0.010397,0.058330,3.661310,-0.118386,2.35739,-0.009112,0.178701,4.097350,3.532890,0.005244,0.121381,0.109968,0.135838,3.46017,0.017054,0.124863,0.154064,0.606848,-0.267928,2.57786,-0.020877,0.024719,0
2,2,0.036330,1.49747,233.5460,2.19435,0.026914,3.12694,5.05687,3.849460,1.80187,0.056995,0.328684,2.968810,0.105244,2.069490,5.30986,1.354790,-0.262018,1.379080,1.48091,0.020542,-0.008806,0.109348,1.68365,0.038180,0.123716,1.11248,3.57166,0.120601,0.082069,2.233520,0.002270,0.045182,0.014405,0.011599,-0.502849,33.7382,1.417500,1.071350,3.22296,...,0.027571,-0.007121,-0.048914,-0.002574,1.865090,2.404170,0.411741,0.057749,0.525174,2.16879,0.828297,0.089848,0.093744,4.949010,-0.010978,0.076671,0.266784,0.038691,0.382731,3.847600,-0.121482,3.74084,0.147098,-0.016566,0.614651,2.125840,0.078828,0.979808,0.026758,0.117310,4.88300,0.085222,0.032396,0.116092,-0.001689,-0.520069,2.14112,0.124464,0.148209,0
3,3,-0.014077,0.24600,779.9670,1.89064,0.006948,1.53112,2.69800,4.517330,4.50332,0.123494,1.002680,4.869600,0.058411,2.497850,1.23843,2.348360,0.175475,1.608890,2.02881,0.042086,0.005141,0.076506,1.65122,0.111813,0.121641,0.58912,4.23692,-0.032843,0.058168,0.712927,0.097465,0.072744,0.000324,0.063362,4.063820,25.3824,0.576572,2.026210,2.96843,...,0.110884,0.026837,2.931160,0.068112,-0.495192,1.345280,2.242750,0.035611,-0.139274,4.74243,3.292740,0.117877,0.065605,0.556711,-0.058029,0.070501,1.101250,0.068559,0.162928,4.070180,-0.008835,3.89678,0.913739,-0.163204,3.074850,4.356780,-0.048894,4.917990,0.069930,-0.015347,3.47439,-0.017103,-0.008100,0.062013,0.041193,0.511657,1.96860,0.040017,0.044873,0
4,4,-0.003259,3.71542,156.1280,2.14772,0.018284,2.09859,4.15492,-0.038236,3.37145,0.034166,0.711483,0.769988,0.057555,0.957257,3.71145,5.464350,0.287104,2.616950,1.38403,0.074883,-0.010543,0.109121,2.27602,0.008023,0.045236,4.35954,5.07562,-0.009376,0.528966,4.053350,0.020000,0.106828,0.051307,0.045939,3.402460,15.5615,1.635960,0.047029,4.01771,...,0.075586,0.032114,-0.042284,0.047974,-0.294184,5.065600,1.050290,0.034019,0.024611,3.12578,2.262840,0.082462,-0.023296,5.615850,0.086238,0.157568,3.725670,0.061247,0.086603,0.607246,1.411090,2.06062,-0.023154,0.011234,2.155530,0.914518,0.044521,0.375731,0.134351,0.013781,1.91059,-0.042943,0.105616,0.125072,0.037509,1.043790,1.07481,-0.012819,0.072798,1


In [ ]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Columns: 102 entries, id to target
dtypes: float64(100), int64(2)
memory usage: 466.9 MB


#2. Prepare train dataset

In [ ]:
X = raw_data.copy()
X.pop('id')
y = X.pop('target')

#3. Model choice

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from numpy.random import uniform

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB

In [ ]:
clf1 = SGDClassifier(n_jobs=-1, random_state=1)
clf2 = LogisticRegression(n_jobs=-1, random_state=1)
clf3 = LinearSVC(dual=False, random_state=1)
clf4 = BernoulliNB()

In [ ]:
pipe = Pipeline([('scaler', StandardScaler()), ('classifier', clf1)])

In [ ]:
params1 = {'classifier__loss' : ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
           'classifier__penalty' : ['l2', 'l1', 'elasticnet'],
           'classifier__alpha' : uniform(0, 1, 1000),
           'classifier__max_iter' : range(1000),
           'classifier__epsilon' : uniform(0, 1, 100),
           'classifier__learning_rate' : ['optimal', 'constant', 'invscaling', 'adaptive'],
           'classifier__eta0' : uniform(0, 1, 100),
           'classifier__power_t' : uniform(0, 1, 100),
           'classifier__class_weight' : [None, 'balanced'],
           'classifier' : [clf1]
           }

params2 = {'classifier__penalty' : ['l1', 'l2', 'elasticnet', 'none'],
           'classifier__tol': uniform(0, 1, 1000),
           'classifier__C': [0.1, 1.0, 10.0, 100.0],
           'classifier__class_weight' : [None, 'balanced'],
           'classifier__solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
           'classifier__multi_class' : ['auto', 'ovr', 'multinomial'],
           'classifier' : [clf2]}

params3 = {'classifier__penalty' : ['l1', 'l2'],
           'classifier__loss' : ['hinge', 'squared_hinge'],
           'classifier__tol': uniform(0, 1, 1000),
           'classifier__C': [0.1, 1.0, 10.0, 100.0],
           'classifier__multi_class' : ['ovr', 'crammer_singer'],
           'classifier__intercept_scaling' : [0.1, 1.0, 10.0, 100.0],
           'classifier__class_weight' : [None, 'balanced'],
           'classifier' : [clf3]
           }

params4 = {'classifier__alpha' : uniform(0, 1, 1000),
           'classifier__binarize' : uniform(0, 1, 1000),
           'classifier' : [clf4]
           }

###a) Best score

In [ ]:
params = [params1, params2, params3, params4]

random_grid = RandomizedSearchCV(pipe, params, n_jobs=-1)
random_grid.fit(X, y)
random_grid.best_estimator_
score = round(random_grid.best_score_, 2)
print(f'Best score: {score * 100}%')

Best score: 73.0%


###b) Best model and parameters

In [ ]:
random_grid.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier',
                 SGDClassifier(alpha=0.9233013057761659,
                               epsilon=0.5206537370817478,
                               eta0=0.5473464220878713, loss='squared_hinge',
                               max_iter=904, n_jobs=-1,
                               power_t=0.590596330557159, random_state=1))])

#4. Predict test values

In [ ]:
test_set = pd.read_csv('test.csv')
test_set.head()

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,...,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
0,600000,0.003229,4.838660,585.529,2.282910,0.713180,3.907830,0.480696,1.482270,4.891810,0.056351,4.200990,3.151800,0.000349,1.85116,2.63889,0.746668,-0.004756,1.610300,4.11482,-0.077756,0.129446,0.053324,0.416789,0.445009,0.150464,5.021300,2.22139,-0.072333,-0.215874,1.56236,0.074880,0.010050,0.018582,0.067466,5.57830,3.08556,3.842470,0.011125,2.35997,...,0.027517,0.048825,0.074626,0.041264,0.069319,4.248810,2.09195,0.695042,0.007464,-0.017882,1.883500,1.268030,0.023747,0.043071,3.174780,0.074356,0.125909,3.926430,0.046914,-0.042290,3.01925,0.089564,3.20070,0.009679,-0.099653,3.573060,4.79727,0.091985,0.773543,0.073380,0.112910,1.073550,0.122149,0.086330,0.036010,0.010619,0.290343,1.898200,0.131533,0.012047
1,600001,0.008602,0.505536,-100.099,3.012670,0.027199,1.194610,5.036620,2.517440,4.553890,0.063876,0.337257,4.439690,0.013188,3.37901,3.38470,1.167400,2.246550,1.750170,2.76624,-0.058501,0.012595,0.036144,0.769057,0.017496,0.050283,0.324697,4.94864,0.124789,0.347128,1.24512,0.035822,-0.013188,0.023194,0.006444,4.98333,23.70690,8.287290,4.796230,1.79928,...,0.067650,0.066438,0.034258,0.606144,0.019988,-0.456704,3.37877,1.900050,0.052316,-0.050397,2.612950,3.112540,0.022305,0.108325,4.617640,0.091440,0.039368,3.035190,1.023050,-0.020578,2.77156,3.309480,5.05840,-0.003993,0.067636,0.995391,2.47165,-0.015214,0.263423,-0.021910,-0.020214,2.622340,0.123307,0.033063,0.123059,0.005771,-0.392923,3.689640,0.047418,0.120015
2,600002,1.461000,2.437260,-112.964,3.541230,0.752338,4.338310,1.648080,4.699910,1.950250,0.005303,2.071680,0.546499,0.141781,1.67317,4.30649,1.702330,-0.062869,1.619230,4.19053,0.055140,-0.016590,0.017805,3.064810,0.070370,0.098316,3.507540,1.06910,0.012750,0.009981,3.46781,0.035920,-0.009804,0.065728,-0.004725,5.28102,11.52880,0.171694,4.394570,2.52084,...,0.065172,-0.003328,-0.001950,0.095007,0.080267,3.337300,3.60489,4.694580,0.075650,-0.088276,1.772440,-0.142226,0.000918,0.021483,0.241844,0.067591,-0.026572,0.894909,0.111606,0.524423,1.45007,0.019355,1.27135,0.076042,0.446993,4.406990,2.44697,-0.041154,0.212414,0.141005,-0.011036,2.030180,-0.000426,0.084091,0.123605,0.499554,4.054650,3.330670,0.108843,0.064687
3,600003,0.140556,3.085610,179.451,0.573945,0.057342,2.216790,1.623480,0.526174,1.542540,-0.026161,1.609440,1.723560,-0.019564,1.55213,4.83264,1.501640,0.192669,4.614890,1.47069,-0.010031,0.072805,0.048035,3.230210,-0.031548,0.028697,3.752520,4.94847,-0.174542,-0.033491,2.47823,0.068129,0.090797,0.029877,0.146718,3.16983,-12.98460,3.135210,1.765010,3.25399,...,0.075775,-0.008900,0.141776,0.173123,0.012894,3.390590,2.69899,0.059916,0.055400,-0.000140,2.641440,0.839721,-0.026268,0.058745,2.521910,0.118737,0.134899,2.839480,0.108751,-0.015848,1.14724,0.390064,3.30923,-0.046117,-0.206052,2.373770,3.15941,0.131234,1.031180,0.026155,0.050117,0.221613,0.045298,0.129966,0.004015,0.018279,2.696580,-0.533491,0.052524,0.011058
4,600004,0.128876,5.199760,107.466,-0.497149,0.080220,0.458121,0.629839,5.240460,-0.232279,0.030006,0.481359,2.176020,0.193162,1.39209,2.51890,2.993170,4.170910,0.318375,4.84563,0.085064,0.026443,-0.004559,0.120327,-0.008630,0.004495,4.921700,1.68564,0.095628,0.189131,2.56955,0.046643,0.111462,0.002912,0.060737,4.25300,38.70340,0.170825,-0.598784,3.92796,...,0.123505,0.011181,0.152410,0.118767,-0.054951,4.841230,3.39522,3.737320,0.053328,0.143619,-0.156581,-0.257201,0.987785,0.038284,2.765120,0.040994,0.173981,1.112190,-0.030675,0.137178,1.55014,0.401083,1.74053,0.096331,0.445268,4.392460,2.52818,-0.037385,0.117148,-0.010128,0.058860,2.660430,0.135425,0.036481,0.093912,0.056315,1.110710,3.584470,0.145319,-0.050393


In [ ]:
test_index = test_set.pop('id')
y_pred = random_grid.predict(test_set)
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
submission_dict = {'id': test_index,
                   'target' : y_pred}

submission_df = pd.DataFrame(submission_dict)
submission_df.head()

,id,target
0,600000,1
1,600001,1
2,600002,1
3,600003,0
4,600004,1


In [ ]:
submission_df.to_csv('submission.csv', index=False)
print('File created')

File created
